<a href="https://colab.research.google.com/github/ZacariDigital/Generative-AI-Absract-Art/blob/main/Generative_A_I_Abstract_ART.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/openai/CLIP
!git clone https://github.com/CompVis/taming-transformers

Cloning into 'CLIP'...
remote: Enumerating objects: 243, done.
remote: Total 243 (delta 0), reused 0 (delta 0), pack-reused 243
Receiving objects: 100% (243/243), 8.92 MiB | 12.26 MiB/s, done.
Resolving deltas: 100% (123/123), done.
Cloning into 'taming-transformers'...
remote: Enumerating objects: 1339, done.
remote: Total 1339 (delta 0), reused 0 (delta 0), pack-reused 1339
Receiving objects: 100% (1339/1339), 409.77 MiB | 16.75 MiB/s, done.
Resolving deltas: 100% (279/279), done.


In [ ]:
##Installing extra libraries
!pip install --no-deps ftfy regex tqdm
!pip install omegaconf==2.0.0 pytorch-lightning==1.0.8
!pip uninstall torchtext --yes
!pip install einops
!pip install torchvision

In [ ]:
!pip install pytorch

In [12]:
##importing libaries
import numpy as np
import torch, os, imageio, pdb, math
import torchvision 
import torchvision.transforms as T
import torchvision.transforms.functional as TF

import PIL 
import matplotlib.pyplot as plt 

import yaml 
from omegaconf import OmegaConf

from CLIP import clip
#import warnings
#warnings.filterwarnings('ignore')

In [15]:
##Coding Helper functions

def show_from_tensor(tensor):
  img = tensor.clone()
  img = img.mul(255).byte()
  img = img.cpu().numpy().transponse((1,2,0))

  plt.figure(figsize=(10,7))
  plt.axis('off')
  plt.imshow(img)
  plt.show()

def norm_data(data):
  return(data.clip(-1,1)+1)/2 ##range between 0 & 1 in the result

##Parameters
learning_rate = .5
batch_size = 1
wd = .1
noise_factor = .1


total_iter=100
im_shape = [225, 400, 3] #h,w,channel
size1, size2, channels = im_shape


In [16]:
## CLIP MODEL ##
clipmodel, _ = clip.load('ViT-B/32', jit=False)
clipmodel.eval()
print(clip.available_models())

print("Clip model visual input resolution", clipmodel.visual.input_resolution)
device=torch.device("cuda:0")
torch.cuda.empty_cache

100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 130MiB/s]


['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14', 'ViT-L/14@336px']
Clip model visual input resolution 224


<function torch.cuda.memory.empty_cache() -> None>

In [18]:
##Taming Transformer##

%cd taming-transformers/

!mkdir -p models/vqgan_imagenet_f16_16384/checkpoints
!mkdir -p models/vqgan_imagenet_f16_16384/configs

if len(os.listdir('models/vqgan_imagenet_f16_16384/configs/')) == 0:
  !wget 'https://heibox.uni-heidelberg.de/f/867b05fc8c4841768640/?dl=1' -O 'models/vqgan_imagenet_f16_16384/checkpoints/last.ckpt'
  !wget 'https://heibox.uni-heidelberg.de/f/274fb24ed38341bfa753/?dl=1' -O 'models/vqgan_imagenet_f16_16384/configs/model.yaml'


[Errno 2] No such file or directory: 'taming-transformers/'
/content/taming-transformers
--2023-05-07 22:26:17--  https://heibox.uni-heidelberg.de/f/867b05fc8c4841768640/?dl=1
Resolving heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)... 129.206.7.113
Connecting to heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)|129.206.7.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://heibox.uni-heidelberg.de/seafhttp/files/f85cf3cc-7219-46e3-a5ba-78927a5f7dde/last.ckpt [following]
--2023-05-07 22:26:18--  https://heibox.uni-heidelberg.de/seafhttp/files/f85cf3cc-7219-46e3-a5ba-78927a5f7dde/last.ckpt
Reusing existing connection to heibox.uni-heidelberg.de:443.
HTTP request sent, awaiting response... 200 OK
Length: 980092370 (935M) [application/octet-stream]
Saving to: ‘models/vqgan_imagenet_f16_16384/checkpoints/last.ckpt’

models/vqgan_imagen 100%[===================>] 934.69M  15.3MB/s    in 64s     

2023-05-07 22:27:22 (14.7 MB/s) - ‘models/vqgan_im